# **Data Cleaning and Preprocessing **

Dataset used: dirty_cafe_sales.csv (https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
path = "/content/drive/MyDrive/dirty_cafe_sales.csv"
df = pd.read_csv(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [3]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_9226047,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [4]:
df.isnull().sum()
df.isnull().mean() * 100   # percentage


,0
Transaction ID,0.00
Item,3.33
Quantity,1.38
Price Per Unit,1.79
Total Spent,1.73
Payment Method,25.79
Location,32.65
Transaction Date,1.59


The Columns 'Payment Method' and 'Location' have more null values and are only categorical, basically no risk on dropping it.

In [5]:
df = df.drop(columns=["Location","Payment Method"])
df.shape

(10000, 6)

In [6]:
df.dtypes


,0
Transaction ID,object
Item,object
Quantity,object
Price Per Unit,object
Total Spent,object
Transaction Date,object


Every column is object, we have to check for numeric values using isnumeric() and change the datatype of necessary columns.

In [7]:
for col in df.columns:
    print(col, df[col].str.isnumeric().mean())


Transaction ID 0.0
Item 0.0
Quantity 0.9654228351247212
Price Per Unit 0.0
Total Spent 0.0
Transaction Date 0.0


In [8]:
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["Price Per Unit"] = pd.to_numeric(df["Price Per Unit"], errors="coerce")
df["Total Spent"] = pd.to_numeric(df["Total Spent"], errors="coerce")
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


In [9]:
df.dtypes

,0
Transaction ID,object
Item,object
Quantity,float64
Price Per Unit,float64
Total Spent,float64
Transaction Date,datetime64[ns]


Successful conversion

In [10]:
df["Item"] = df["Item"].str.strip().str.title()


Standartize the names

In [11]:
df["Item"].value_counts()
df.groupby("Item")["Price Per Unit"].mean()


,Price Per Unit
Item,
Cake,3.000000
Coffee,2.000000
Cookie,1.000000
Error,2.944444
Juice,3.000000
Salad,5.000000
Sandwich,4.000000
Smoothie,4.000000
Tea,1.500000


Mapping of the Items to the Unit Price.

In [12]:
df.describe()

,Quantity,Price Per Unit,Total Spent,Transaction Date
count,9521.000000,9467.000000,9498.000000,9540
mean,3.028463,2.949984,8.924352,2023-07-01 23:00:31.698113280
min,1.000000,1.000000,1.000000,2023-01-01 00:00:00
25%,2.000000,2.000000,4.000000,2023-04-01 00:00:00
50%,3.000000,3.000000,8.000000,2023-07-02 00:00:00
75%,4.000000,4.000000,12.000000,2023-10-02 00:00:00
max,5.000000,5.000000,25.000000,2023-12-31 00:00:00
std,1.419007,1.278450,6.009919,NaN


In [13]:
df.describe(include="object")


,Transaction ID,Item
count,10000,9667
unique,10000,10
top,TXN_9226047,Juice
freq,1,1171


Statistical analysis to find improper data using distribution,deviation,outliers,etc.

In [14]:
df.skew(numeric_only=True)

,0
Quantity,-0.008093
Price Per Unit,0.004591
Total Spent,0.823826


In [15]:
df.corr(numeric_only=True)


,Quantity,Price Per Unit,Total Spent
Quantity,1.000000,0.006078,0.704310
Price Per Unit,0.006078,1.000000,0.647489
Total Spent,0.704310,0.647489,1.000000


In [16]:
df.mean(numeric_only=True)


,0
Quantity,3.028463
Price Per Unit,2.949984
Total Spent,8.924352


In [17]:

df.median(numeric_only=True)

,0
Quantity,3.0
Price Per Unit,3.0
Total Spent,8.0


In [18]:
df.std(numeric_only=True)


,0
Quantity,1.419007
Price Per Unit,1.278450
Total Spent,6.009919


In [19]:
df.var(numeric_only=True)

,0
Quantity,2.013581
Price Per Unit,1.634436
Total Spent,36.119132


In [20]:
df.kurtosis(numeric_only=True)

,0
Quantity,-1.312557
Price Per Unit,-1.155705
Total Spent,-0.145201


In [21]:
df["Item"].value_counts()


,count
Item,
Juice,1171
Coffee,1165
Salad,1148
Cake,1139
Sandwich,1131
Smoothie,1096
Cookie,1092
Tea,1089
Unknown,344


Fill the empty rows in 'Item' and 'Price Per Unit' by mapping each other.

In [22]:
import numpy as np

df["Item"] = df["Item"].replace(["ERROR", "UNKNOWN", ""], np.nan)


In [23]:
item_to_price = (
    df.dropna(subset=["Item", "Price Per Unit"])
      .groupby("Item")["Price Per Unit"]
      .agg(lambda x: x.mode()[0])
)

price_to_item = (
    df.dropna(subset=["Item"])
      .groupby("Price Per Unit")["Item"]
      .agg(lambda x: x.mode()[0])
)

In [24]:
mask = df["Item"].isna() & df["Price Per Unit"].notna()
df.loc[mask, "Item"] = df.loc[mask, "Price Per Unit"].map(price_to_item)

mask = df["Price Per Unit"].isna() & df["Item"].notna()
df.loc[mask, "Price Per Unit"] = df.loc[mask, "Item"].map(item_to_price)


In [25]:
df[["Item", "Price Per Unit"]].isna().sum()


,0
Item,23
Price Per Unit,23


Fill the rows of 'Price Per Unit', 'Quantity' and 'Total Spent' by applying the logic:

Price Per Unit x Quantity = Total Spent.

In [26]:
import numpy as np

df.replace(["ERROR", "error", "unknown", "UNKNOWN",""], np.nan, inplace=True)


In [27]:
cols = ["Quantity", "Price Per Unit", "Total Spent"]
missing_count = df[cols].isna().sum(axis=1)

print(missing_count)


0       0
1       0
2       1
3       0
4       0
       ..
9995    0
9996    1
9997    0
9998    0
9999    0
Length: 10000, dtype: int64


In [28]:
rows_before = len(df)

df = df[missing_count < 2].copy()

rows_after = len(df)
rows_dropped = rows_before - rows_after
print(df)
print(f"Number of rows dropped: {rows_dropped}")

     Transaction ID      Item  Quantity  Price Per Unit  Total Spent  \
0       TXN_1961373    Coffee       2.0             2.0          4.0   
1       TXN_4977031      Cake       4.0             3.0         12.0   
2       TXN_4271903    Cookie       4.0             1.0          NaN   
3       TXN_7034554     Salad       2.0             5.0         10.0   
4       TXN_3160411    Coffee       2.0             2.0          4.0   
...             ...       ...       ...             ...          ...   
9995    TXN_7672686    Coffee       2.0             2.0          4.0   
9996    TXN_9659401       NaN       3.0             NaN          3.0   
9997    TXN_5255387    Coffee       4.0             2.0          8.0   
9998    TXN_7695629    Cookie       3.0             1.0          3.0   
9999    TXN_6170729  Sandwich       3.0             4.0         12.0   

     Transaction Date  
0          2023-09-08  
1          2023-05-16  
2          2023-07-19  
3          2023-04-27  
4          2023

In [29]:
# Fill Total
mask = df["Total Spent"].isna()
df.loc[mask, "Total Spent"] = (
    df.loc[mask, "Quantity"] * df.loc[mask, "Price Per Unit"]
)

# Fill Price
mask = df["Price Per Unit"].isna()
df.loc[mask, "Price Per Unit"] = (
    df.loc[mask, "Total Spent"] / df.loc[mask, "Quantity"]
)

# Fill Quantity
mask = df["Quantity"].isna()
df.loc[mask, "Quantity"] = (
    df.loc[mask, "Total Spent"] / df.loc[mask, "Price Per Unit"]
)
df[cols].isna().sum()


,0
Quantity,0
Price Per Unit,0
Total Spent,0


Check for invalid row which doesn't follow the logic.

In [30]:
invalid_rows = df[
    df["Total Spent"] != df["Quantity"] * df["Price Per Unit"]
]

len(invalid_rows)

19

Check for Outliers.

In [31]:
Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1
outliers = (df["Price Per Unit"] < Q1["Price Per Unit"] - 1.5*IQR["Price Per Unit"]) | \
           (df["Price Per Unit"] > Q3["Price Per Unit"] + 1.5*IQR["Price Per Unit"])
print(df[outliers])


Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Transaction Date]
Index: []


In [32]:
from scipy.stats import zscore

df["price_z"] = zscore(df["Price Per Unit"])
print(df[df["price_z"].abs() > 3])


Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Transaction Date, price_z]
Index: []


In [33]:
df.drop(columns=["price_z"], inplace=True)


Drop the duplicates.

In [34]:
df = df.drop_duplicates()
df.shape

(9978, 6)

Apply filtering logics to remove negative price, negative quantity, improper dates, future dates.

In [35]:
before = len(df)

df = df[
    (df["Price Per Unit"] > 0) &
    (df["Quantity"] > 0) &
    (df["Transaction Date"].notna()) &
    (df["Transaction Date"] <= pd.Timestamp.today())
]

removed_rows = before - len(df)
print("Rows removed:", removed_rows)


Rows removed: 460


In [36]:
df.groupby("Item")["Total Spent"].mean()


,Total Spent
Item,
Cake,9.097132
Coffee,6.083045
Cookie,2.974743
Error,8.830389
Juice,8.939698
Salad,15.079435
Sandwich,12.222997
Smoothie,12.214149
Tea,4.539698


Final Output:

In [37]:
print(df.to_string())
print(df.shape)

     Transaction ID      Item  Quantity  Price Per Unit  Total Spent Transaction Date
0       TXN_1961373    Coffee  2.000000             2.0          4.0       2023-09-08
1       TXN_4977031      Cake  4.000000             3.0         12.0       2023-05-16
2       TXN_4271903    Cookie  4.000000             1.0          4.0       2023-07-19
3       TXN_7034554     Salad  2.000000             5.0         10.0       2023-04-27
4       TXN_3160411    Coffee  2.000000             2.0          4.0       2023-06-11
5       TXN_2602893  Smoothie  5.000000             4.0         20.0       2023-03-31
6       TXN_4433211   Unknown  3.000000             3.0          9.0       2023-10-06
7       TXN_6699534  Sandwich  4.000000             4.0         16.0       2023-10-28
8       TXN_4717867     Juice  5.000000             3.0         15.0       2023-07-28
9       TXN_2064365  Sandwich  5.000000             4.0         20.0       2023-12-31
10      TXN_2548360     Salad  5.000000             5.

In [38]:
df.to_csv("cleaned_cafe_sales.csv", index=False)
from google.colab import files
files.download("cleaned_cafe_sales.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>